# Создание вопросно-ответного приложения на основе графовой базы данных

В разделе рассмотрены основные способы создания цепочки Вопрос-Ответ на основе графовой базы данных. Такие системы позволят нам задавать вопросы о данных в графовой базе данных и получать ответы на естественном языке.

:::danger

Вопросно-ответные приложения на основе графовых баз данных требуют выполнения SQL запросов, которые сгенерировала модель.
Это связано с определенными рисками.
Убедитесь, что вашим цепочкам/агентам предоставлены только необходимые разрешения.
Это снизит, но не устранит полностью, риски создания системы, управляемой моделью.
Подробнее — в разделе [Безопасность](/docs/security).

:::

## Архитектура

В общем случае работу таких приложений можно представить этапами:

1. Преобразование вопроса в запрос к графовой базе данных. Модель преобразует ввод пользователя в запрос к графовой базе данных (например, Cypher).
2. Выполнение запроса к графовой базе данных.
3. Ответ на вопрос. Модель отвечает на ввод пользователя, используя результаты запроса.

<Image img={require('../../static/img/graph_usecase.png')} alt="sql_usecase.png" />

## Подготовка к работе

Установите необходимые пакеты и задайте переменные окружения.
В представленном примере используется графовая база данных Neo4j.

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai neo4j

При работе с этим руководством используется модель OpenAI.

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Uncomment the below to use LangSmith. Not required.
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

 ········


После этого нужно задать учетные данные Neo4j.
Следуйте [инструкции по установке](https://neo4j.com/docs/operations-manual/current/installation/), чтобы настроить базу данных Neo4j.

In [2]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

Пример ниже создает подключение к базе данных Neo4j и заполняет ее тестовыми данными о фильмах и их актерах.

In [3]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

# Импорт информации о фильме

movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)

[]

## Схема графа

Чтобы модель могла генерировать Cypher-запросы, ей нужна информация о схеме графа.
Схема возвращается при создании объекта графа.
Если позже вы внесете изменения в граф, то сможете использовать метод `refresh_schema`, чтобы обновить информацию о схеме.

In [4]:
graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Movie {imdbRating: FLOAT, id: STRING, released: DATE, title: STRING},Person {name: STRING},Genre {name: STRING},Chunk {id: STRING, question: STRING, query: STRING, text: STRING, embedding: LIST}
Relationship properties are the following:

The relationships are the following:
(:Movie)-[:IN_GENRE]->(:Genre),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:ACTED_IN]->(:Movie)


После создания базы данных попробуйте подключить ее к LLM.

## Цепочка

Вы можете создать простую цепочку, которая принимает вопрос и выполняет действия:

- преобразует вопрос в Cypher-запрос;
- выполняет запрос;
- использует результат для ответа на исходный вопрос.

![graph_chain.webp](../../static/img/graph_chain.webp)

Для реализации такого процесс в GigaChain есть встроенная цепочка [`GraphCypherQAChain`](/docs/integrations/graphs/neo4j_cypher), которая работает с Neo4j.

In [5]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What was the cast of the Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Movie {title: "Casino"})<-[:ACTED_IN]-(actor:Person)
RETURN actor.name
Full Context:
[{'actor.name': 'Joe Pesci'}, {'actor.name': 'Robert De Niro'}, {'actor.name': 'Sharon Stone'}, {'actor.name': 'James Woods'}]

> Finished chain.


{'query': 'What was the cast of the Casino?',
 'result': 'The cast of Casino included Joe Pesci, Robert De Niro, Sharon Stone, and James Woods.'}

# Проверка направления связей

LLM могут испытывать трудности с направлениями связей в сгенерированных Cypher-запросах.
Поскольку схема графа задана заранее, вы можете проверять и, если нужно, корректировать направления связей в сгенерированных Cypher-запросах, с помощью параметра `validate_cypher`.

In [6]:
chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True, validate_cypher=True
)
response = chain.invoke({"query": "What was the cast of the Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Movie {title: "Casino"})<-[:ACTED_IN]-(actor:Person)
RETURN actor.name
Full Context:
[{'actor.name': 'Joe Pesci'}, {'actor.name': 'Robert De Niro'}, {'actor.name': 'Sharon Stone'}, {'actor.name': 'James Woods'}]

> Finished chain.


{'query': 'What was the cast of the Casino?',
 'result': 'The cast of Casino included Joe Pesci, Robert De Niro, Sharon Stone, and James Woods.'}

## Смотрите также

* [Стратегии создания промптов](/docs/how_to/graph_prompting)
* [Соответствие значений](/docs/how_to/graph_mapping): методики сопоставления значений из вопросов к базе данных.
* [Семантический слой](/docs/how_to/graph_semantic): методики работы с семантическими слоями.
* [Построение графов](/docs/how_to/graph_constructing): методики построения графов знаний.